In [15]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate

In [55]:
# Étape 1 : Charger les documents PDF
import os
from PyPDF2 import PdfReader

def load_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            with open(os.path.join(folder_path, filename), 'rb') as file:
                reader = PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
                documents.append(text)  # Ajouter le texte extrait à la liste
    return documents


In [59]:
documents = load_documents(r"C:\Users\maserati\Desktop\rag_llm")
print("Documents chargés :", documents)


Documents chargés : ["Certificat de formation  de juge de Double Dutch  \n \n \n \nNous, soussignés, certifions que Assa Traoré  a suivi avec succès la formation de Juge de \nDouble Dutch  organisée par la Fédération Française de Double Dutch  en 2020 . \nCette formation couvre l'évaluation des compétences nécessaires pour juger les compétitions \nde Double Dutch, y compris la compréhension des règles, des critères de jugement et des \nprocédures officielles. Par la présente, nous attestons que Assa Traoré  a acquis les \ncompétences requises pour exercer en tant que juge dans ce domaine.  \nFait à Vitry -sur-Seine  le 3 décembre 2020 . \n \n \n \n \n \n \nNom  : Jonathan Mahoto  \nPoste  : Directeur de la FFDD  \n \n                                                               \n \n \n \n                                                                                                                                       \n", "Assa Traoré\nData analy st\nMaster MASERA TI\nUPE C Crétei

In [65]:
import re

def clean_documents(documents):
    """
    Nettoie les documents en supprimant les caractères inutiles et les espaces excessifs.
    """
    cleaned_documents = []
    
    for doc in documents:
        # Nettoyage du texte
        cleaned_text = re.sub(r'\n+', ' ', doc)  # Remplace les retours à la ligne multiples par un espace
        cleaned_text = re.sub(r'\uf0e0|\uf015|\uf073|\uf109|\uf2c3|\uf1b9|\uf041|\uf098', '', cleaned_text)  # Supprime les caractères spéciaux
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Remplace les espaces multiples par un seul espace
        cleaned_text = cleaned_text.strip()  # Supprime les espaces en début et fin de texte
        
        # Ajouter le texte nettoyé à la liste
        cleaned_documents.append(cleaned_text)
    
    return cleaned_documents

# Nettoyer les documents après les avoir chargés
cleaned_documents = clean_documents(documents)
print("Documents nettoyés :", cleaned_documents)


Documents nettoyés : ["Certificat de formation de juge de Double Dutch Nous, soussignés, certifions que Assa Traoré a suivi avec succès la formation de Juge de Double Dutch organisée par la Fédération Française de Double Dutch en 2020 . Cette formation couvre l'évaluation des compétences nécessaires pour juger les compétitions de Double Dutch, y compris la compréhension des règles, des critères de jugement et des procédures officielles. Par la présente, nous attestons que Assa Traoré a acquis les compétences requises pour exercer en tant que juge dans ce domaine. Fait à Vitry -sur-Seine le 3 décembre 2020 . Nom : Jonathan Mahoto Poste : Directeur de la FFDD", "Assa Traoré Data analy st Master MASERA TI UPE C Créteil Logiciel e t langag e de pr ogramma tion Séries t empor elles Économé trie Analyse de données DU - I ntelligence Artiﬁcielle UPE C Créteil Machine L earning Deep L earning Spark Logique e t programma tion Algorithme a vancée Licence économie e t gestion UPE C Créteil Microé

In [70]:
# Étape 2 : Fractionner les documents en chunks
from langchain.schema import Document

def create_documents(texts):
    """
    Crée des objets Document à partir des textes nettoyés.
    """
    documents2 = []
    for text in texts:
        doc = Document(page_content=text, metadata={})  # Vous pouvez ajouter des métadonnées si nécessaire
        documents2.append(doc)
    return documents2


In [79]:
documents3=create_documents(cleaned_documents)
documents3

[Document(metadata={}, page_content="Certificat de formation de juge de Double Dutch Nous, soussignés, certifions que Assa Traoré a suivi avec succès la formation de Juge de Double Dutch organisée par la Fédération Française de Double Dutch en 2020 . Cette formation couvre l'évaluation des compétences nécessaires pour juger les compétitions de Double Dutch, y compris la compréhension des règles, des critères de jugement et des procédures officielles. Par la présente, nous attestons que Assa Traoré a acquis les compétences requises pour exercer en tant que juge dans ce domaine. Fait à Vitry -sur-Seine le 3 décembre 2020 . Nom : Jonathan Mahoto Poste : Directeur de la FFDD"),
 Document(metadata={}, page_content="Assa Traoré Data analy st Master MASERA TI UPE C Créteil Logiciel e t langag e de pr ogramma tion Séries t empor elles Économé trie Analyse de données DU - I ntelligence Artiﬁcielle UPE C Créteil Machine L earning Deep L earning Spark Logique e t programma tion Algorithme a vancé

In [72]:
# Étape 2 : Fractionner les documents en chunks
def split_documents(documents):
    """
    Fractionne les documents en chunks pour l'indexation.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_documents(documents)

# Création des documents à partir du texte nettoyé
documents = create_documents(cleaned_documents)

# Fractionner les documents
chunks = split_documents(documents)

# Afficher les chunks créés
print("Chunks créés :")
for chunk in chunks:
    print(chunk.page_content)  # Afficher le contenu de chaque chunk

Chunks créés :
Certificat de formation de juge de Double Dutch Nous, soussignés, certifions que Assa Traoré a suivi avec succès la formation de Juge de Double Dutch organisée par la Fédération Française de Double Dutch en 2020 . Cette formation couvre l'évaluation des compétences nécessaires pour juger les compétitions de Double Dutch, y compris la compréhension des règles, des critères de jugement et des procédures officielles. Par la présente, nous attestons que Assa Traoré a acquis les compétences requises pour exercer en tant que juge dans ce domaine. Fait à Vitry -sur-Seine le 3 décembre 2020 . Nom : Jonathan Mahoto Poste : Directeur de la FFDD
Assa Traoré Data analy st Master MASERA TI UPE C Créteil Logiciel e t langag e de pr ogramma tion Séries t empor elles Économé trie Analyse de données DU - I ntelligence Artiﬁcielle UPE C Créteil Machine L earning Deep L earning Spark Logique e t programma tion Algorithme a vancée Licence économie e t gestion UPE C Créteil Microéconomie Mac

In [73]:
# Étape 3 : Créer un index vectoriel avec Cohere
from langchain_community.embeddings.cohere import CohereEmbeddings

def create_vectorstore(documents, cohere_api_key):
    """
    Crée un index vectoriel à partir des documents et des embeddings Cohere.
    """
    embeddings = CohereEmbeddings(
        cohere_api_key=cohere_api_key,
        model="embed-english-v2.0",
        user_agent="custom_user_agent"  # Ajout explicite du champ
    )
    vectorstore = Chroma.from_documents(documents, embeddings)
    return vectorstore


In [74]:
cohere_api_key = 'IiVPAkCJc5XsF4abBihOzkj2I7WcYKVH4KgLZMID'  
cohere_client = cohere.Client(api_key=cohere_api_key)

In [80]:
create_vectorstore(documents3, cohere_api_key)

In [82]:
# Étape 4 : Définir un prompt pour générer un CV
def define_prompt():
    """
    Définir un modèle de prompt pour générer un CV structuré.
    """
    template = """
    Vous êtes un assistant intelligent spécialisé dans la rédaction de CV.
    Utilisez les informations suivantes pour générer un CV professionnel structuré :

    Informations :
    {context}

    Structure du CV :
    1. Informations personnelles
    2. Expérience professionnelle
    3. Compétences
    4. Certifications et formations
    5. Projets

    Résultat attendu : 
    - Un CV bien structuré et rédigé de manière claire et professionnelle.
    """
    return PromptTemplate(template=template, input_variables=["context"])

In [83]:
# Étape 5 : Interroger le modèle et générer le CV
def generate_cv(query, vectorstore, cohere_api_key):
    """
    Génère un CV en utilisant un LLM et un vecteur de recherche.
    """
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(query)

    # Récupérer le contexte pour le LLM
    context = "\n\n".join([doc.page_content for doc in docs])

    # Configurer le LLM Cohere
    llm = Cohere(cohere_api_key=cohere_api_key, model="command-xlarge-nightly")

    # Créer un prompt
    prompt = define_prompt()
    formatted_prompt = prompt.format(context=context)

    # Générer le résultat
    result = llm(formatted_prompt)
    return result


In [84]:
generate_cv

<function __main__.generate_cv(query, vectorstore, cohere_api_key)>

In [86]:
# Étape 6 : Intégrer le tout
if __name__ == "__main__":
    # Configurer la clé API Cohere
    COHERE_API_KEY = "IiVPAkCJc5XsF4abBihOzkj2I7WcYKVH4KgLZMID"

    # Chemin du dossier contenant les CV et certificats
    input_folder = r"C:\Users\maserati\Desktop\rag_llm"

    # Charger et traiter les documents
    documents = load_documents(input_folder)
    chunks = split_documents(documents3)

    # Créer un vectorstore
    vectorstore = create_vectorstore(chunks, COHERE_API_KEY)

    # Générer un CV basé sur une requête utilisateur
    query = "Créer un CV basé sur mes expériences et certificats"
    generated_cv = generate_cv(query, vectorstore, COHERE_API_KEY)

    # Afficher le CV généré
    print("CV Généré :\n")
    print(generated_cv)

C:\Users\maserati\AppData\Local\Temp\ipykernel_14196\379781450.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)
C:\Users\maserati\AppData\Local\Temp\ipykernel_14196\379781450.py:13: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere(cohere_api_key=cohere_api_key, model="command-xlarge-nightly")
C:\Users\maserati\AppData\Local\Temp\ipykernel_14196\379781450.py:20: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` ins

CV Généré :

# CV d'Assa Traoré

## Informations personnelles

- Adresse e-mail: traore.assa16@gmail.com
- Téléphone: 07 69 88 07 32
- Date de naissance: 16/01/2004
- Adresse: Grigny (91350), France
- Permis: B1
- Véhicule: Personnel

## Expérience professionnelle

- **Équipière polyvalente, McDonald's Grigny (juin 2023 - avril 2024):**
  - Inventaire et contrôle des stocks
  - Application des procédures de fermeture
  - Préparation de commande

- **Préparatrice de commande, Amazon Brétigny-sur-Orge (avril 2022 - avril 2023):**
  - Contrôle de la qualité
  - Suivi des niveaux de stocks
  - Préparation de commandes

- **Hôtesse de caisse, Bricoman Monthléry (septembre 2021 - avril 2022):**
  - Encaissement de la clientèle
  - Gestion des


In [88]:
def generate_cv2(query, vectorstore, cohere_api_key):
    """
    Utilise un LLM pour générer un CV basé sur la requête et les documents.
    """
    retriever = vectorstore.as_retriever()
    prompt = PromptTemplate(
        input_variables=["context", "query"],
        template=(
            "En se basant sur les informations suivantes : {context}\n"
            "Répondez à la requête suivante : {query}"
        )
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=CohereEmbeddings(cohere_api_key=cohere_api_key),
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False
    )
    return qa_chain.run(query)

if __name__ == "__main__":
    # Configurer la clé API Cohere
    COHERE_API_KEY = "IiVPAkCJc5XsF4abBihOzkj2I7WcYKVH4KgLZMID"

    # Chemin du dossier contenant les CV et certificats
    input_folder = r"C:\Users\maserati\Desktop\rag_llm"

    # Charger et traiter les documents
    documents = load_documents(input_folder)
    if not documents:
        raise ValueError("Aucun document n'a été chargé.")
    chunks = split_documents(documents3)

    # Créer un vectorstore
    vectorstore = create_vectorstore(chunks, COHERE_API_KEY)

    # Générer un CV basé sur une requête utilisateur
    query = "Créer un CV basé sur mes expériences et certificats"
    generated_cv = generate_cv(query, vectorstore, COHERE_API_KEY)

    # Afficher le CV généré
    print("CV Généré :\n")
    print(generated_cv)

CV Généré :

# CURRICULUM VITAE

## Assa Traoré

[**Téléphone**] 07 69 88 07 32
[**Adresse e-mail**] traore.assa16@gmail.com
[**Adresse postale**] Grigny (91350), France

## Profil professionnel

Je suis un analyste de données motivé, actuellement inscrit au Master MASERATI à l'UPEC Créteil, avec une passion pour l'intelligence artificielle, le machine learning et l'analyse de données. J'ai une expérience pratique dans la gestion des données, la programmation et la résolution de problèmes analytiques complexes. Je cherche à entrer dans le domaine des données pour appliquer mes compétences et contribuer de manière significative à la prise de décisions fondée sur les données.

## Expérience professionnelle

### Équipière polyvalente, McDonald's Grigny (septembre 2021 - avril 2022)
- J'ai participé à différentes tâches de restauration, notamment la préparation des commandes, l'inventaire des stocks et l'application des procédures de fermeture.

### Aide aux devoirs, Mairie de Grigny (nov


In [138]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import CohereEmbeddings

def generate_cv2(query, vectorstore, cohere_api_key, temperature=0.7, max_tokens=2000):
    """
    Utilise un LLM pour générer un CV basé sur la requête et les documents,
    avec des options de personnalisation pour la température et le nombre de tokens.
    """
    # Créer un retriever pour interroger le vectorstore
    retriever = vectorstore.as_retriever()

    # Définir le prompt qui sera utilisé pour la génération du texte
    prompt = PromptTemplate(
        input_variables=["context", "query"],
        template=(
            "En se basant sur les informations suivantes : {context}\n"
            "Répondez à la requête suivante : {query}"
        )
    )

    # Ajouter le user_agent
    user_agent = "mon_client"  # Choisissez un nom pour le client

    # Créer les embeddings avec le user_agent
    cohere_embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, user_agent=user_agent)

    # Créer le modèle Cohere pour la génération de texte
    llm = Cohere(
        cohere_api_key=cohere_api_key,  # Utilisation de la clé API directement ici
        model="command-xlarge",  # Spécifiez le modèle de génération approprié
        temperature=temperature,
        max_tokens=max_tokens
    )

    # Créer la chaîne de question-réponse avec les paramètres de température et de tokens
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Cette méthode combine les documents
        retriever=retriever,
        return_source_documents=False
    )

    # Exécuter la chaîne pour obtenir la réponse
    response = qa_chain.run(query)

    return response


# Exemple d'utilisation de la fonction avec des paramètres modifiables
if __name__ == "__main__":
    # Configurer la clé API Cohere
    COHERE_API_KEY = "IiVPAkCJc5XsF4abBihOzkj2I7WcYKVH4KgLZMID"

    # Exemple de documents fictifs (remplacez-les par vos vrais documents)
    documents = ["Voici une expérience de travail.", "Voici une certification acquise."]

    # Créer un vectorstore avec FAISS et CohereEmbeddings
    embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY, user_agent="mon_client")
    vectorstore = FAISS.from_documents(documents3, embeddings)

    # Générer un CV basé sur une requête utilisateur avec des paramètres de température et de tokens
    query = '''Créer un CV basé sur mes expériences et certificats en respectant ce plan :
    1- Information 
    2 - Education 
    3- Certifications
    4 - Skills
    5 - Work Experience
    6 - Projects/Internships/Volunteer Experience '''
    generated_cv = generate_cv2(query, vectorstore, COHERE_API_KEY, temperature=0.7, max_tokens=2000)

    # Afficher le CV généré en Markdown
    print("\n### CV Généré :\n")
    markdown_cv = f"""
    # Curriculum Vitae

    {generated_cv}
    """
    display(Markdown(generated_cv))


### CV Généré :



 Sure, I can help you create a CV based on the information you've provided. Here's a draft of a CV in respect of the plan you provided;

Information:
- Full Name: Assa Traoré
- Date of Birth: January 16, 2004
- Languages: French, English​TOEIC: 731/990-, Spanish

Education:
- Master of Data Science and Artificial Intelligence | UPE C & Créteil, 2023
- License in Economic and Management | UPE C & CRETIL, 2023 
- Master of Machine Learning and Deep Learning | Spark and Logique et Programmation, 2022
- Master of Data Economics and Analytics | Master's Degree in Data Science and Artificial Intelligence, 2022
- Bachelor's degree in Economic and Management | Lycée Jean-Baptiste Corot, Savigny-sur-Orge, 2020

Certifications:
- Juror of Double Dutch Certification | Federation Française de Double Dutch, 2020

Skills:
- Advanced knowledge of data science and artificial intelligence tools and platforms
- Proficient in programming languages such as Spark and Logique
- Experience with machine learning algorithms and deep learning methodologies
- Expertise in macro and microeconomics, stochastic processes, and financial economics
- Strong analytical and problem-solving skills
- Ability to work independently and as part of a team
- Excellent communication skills in French and English

Work Experience:
- Data Science and Artificial Intelligence Intern | UPE C & Créteil, 2023
- Machine Learning and Deep Learning Intern | Spark, 2022
- Economic and Management Intern | Lycée Jean-Baptiste Corot, Savigny-sur-Orge, 2020

Projects/Internships/Volunteer Experience:
- Grigny McDonald's | Crew Member, 2024
- Bricoman Monthlery | Sales Assistant, 2022
- No Limit Jump | Assistant Event Coordinator, 2020

In respect of your criteria, I have laid out the given information in the respective sections of the CV. I remain hesitant to make any alterations to the information you've provided me with. 

In [137]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import CohereEmbeddings

def generate_cv2(query, vectorstore, cohere_api_key, temperature=0.4, max_tokens=2000):
    """
    Utilise un LLM pour générer un CV basé sur la requête et les documents,
    avec des options de personnalisation pour la température et le nombre de tokens.
    """
    # Créer un retriever pour interroger le vectorstore
    retriever = vectorstore.as_retriever()

    # Définir le prompt qui sera utilisé pour la génération du texte
    prompt = PromptTemplate(
        input_variables=["context", "query"],
        template=(
            "En se basant sur les informations suivantes : {context}\n"
            "Répondez à la requête suivante : {query}"
        )
    )
    # Ajouter le user_agent
    user_agent = "mon_client"  # Choisissez un nom pour le client

    # Créer les embeddings avec le user_agent
    cohere_embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, user_agent=user_agent)

    # Créer le modèle Cohere pour la génération de texte
    llm = Cohere(
        cohere_api_key=cohere_api_key,  # Utilisation de la clé API directement ici
        model="command-xlarge",  # Spécifiez le modèle de génération approprié
        temperature=temperature,
        max_tokens=max_tokens
    )

    # Créer la chaîne de question-réponse avec les paramètres de température et de tokens
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Cette méthode combine les documents
        retriever=retriever,
        return_source_documents=False
    )
    # Exécuter la chaîne pour obtenir la réponse
    response = qa_chain.run(query)

    return response


# Exemple d'utilisation de la fonction avec des paramètres modifiables
if __name__ == "__main__":
    # Configurer la clé API Cohere
    COHERE_API_KEY = "IiVPAkCJc5XsF4abBihOzkj2I7WcYKVH4KgLZMID"

    # Créer un vectorstore avec FAISS et CohereEmbeddings
    embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY, user_agent="mon_client")
    vectorstore = FAISS.from_documents(documents3, embeddings)

    # Générer un CV basé sur une requête utilisateur avec des paramètres de température et de tokens
    query = '''Créer un CV basé sur mes expériences et certificats en respectant ce plan :
    1- Information 
    2 - Education 
    3- Certifications
    4 - Skills
    5 - Work Experience
    6 - Projects/Internships/Volunteer Experience '''
    generated_cv = generate_cv2(query, vectorstore, COHERE_API_KEY, temperature=0.7, max_tokens=2000)

    # Afficher le CV généré en Markdown
    print("\n### CV Généré :\n")
    markdown_cv = f"""
    # Curriculum Vitae

    {generated_cv}
    """
    display(Markdown(generated_cv))


### CV Généré :



 Sure, I can help you create a basic CV based on the information you've provided. Here's a draft of how such a document could be structured:

---

**Assa Traore**

Email: assa16@gmail.com 

Phone: (07 69 88 07 32) 

---

# INFORMATION
- Date of Birth: 16/01/2004
- Languages: English (& TOEIC score 731/990), Spanish
- Residential Address: Grigny (91350) 

# EDUCATION
- Master of Data Analysis, UPE C & M Créteil
- Licence économie et gestion, UPE C & M Créteil
- Master of Artificielle Intelligence, UPE C & M Créteil
- Master of Machine Learning, UPE C & M Créteil
- Master of Deep Learning, UPE C & M Créteil
- Master of Algorithmic Innovation, UPE C & M Créteil
- Master of Logiciel Language, UPE C & M Créteil

# CERTIFICATIONS
- Cert. in Economic Sciences, Lycee Jean-Baptiste Corot Savigny-sur-Orge
- Cert. in Mathematical, Statistical and Economic Sciences, Lycee Jean-Baptiste Corot Savigny-sur-Orge
- Cert. in TOEIC, Lycee Jean-Baptiste Corot Savigny-sur-Orge
- Cert. in Double Dutch Judging, Federation Française de Double Dutch (FFDD), 2020

# SKILLS
- Data analytics and statistical analysis
- Economic analysis and finance comprehension
- Project management and event planning
- Language interpretation and translation
- Software engineering (Java, Python, Spark) 
- Artificial intelligence and machine learning 
- Deep learning algorithm design and deployment 

# WORK EXPERIENCE

## Economic and Financial Analyst, DU - Intelligence Artificielle, UPE C & M Créteil
- Analyzed large datasets to identify trends and patterns in economic and financial data
- Developed predictive models and conducted risk assessments to support strategic decision-making
- Collaborated with cross-functional teams to design and implement data-driven solutions for clients

## Maire de Grigny, Grigny

### Aide aux de voirs

- Assisted with administrative tasks related to the municipality's economic development initiatives

### Mise en place des événements sportifs et culturels

- Planned and organized sports and cultural events for children unable to attend summer holidays due to the COVID-19 pandemic

## No Limit Jump, Grigny

### Aide aux révisions 

- Assisted students with lesson revisions and provided supplementary exercises

### Apprentissage des leçons 

- Assisted with lesson preparation and delivered classroom support

## Hôtesse de caisse, Bricoman Monthlery

### Encaissement de la clientèle

- Managed cash register operations, including inventory and control of stocks

### Gestion des retours 

- Managed return processes and ensured high levels of customer satisfaction 

## Conseillère clientèle, Amazon Brétigny-sur-Orge

### Contrôle de la qualité

- Ensured consistent quality control of stocks and monitored water levels

### Suivi de niveaux de stocks 

- Conducted inventory checks and organized stock storage

### Préparation des commandes

- Prepared orders for shipping and managed customer orders 

## Spécialiste Math, SES, HGGSP, Lycee Jean-Baptiste Corot Savigny-sur-Orge 

### Service Civique

- Taught mathematics to students and assisted with homework

## Licenciée en Microéconomie, Macroéconomie, Processus Stochastique, UPE C & M Créteil 

### Économétrie

- Analyzed economic data to forecast market trends and consumer behaviour 

### Financé

- Evaluated financial statements and assessed corporate investment strategies 

# PROJECTS/INTERNSHIPS/VOLUNTEER EXPERIENCE

## Master of Licence économie et gestion, UPE C & M Créteil

###Dissertation:  "Title of Dissertation" (Expected Graduation: 2024)

## TOEIC Certification, Lycee Jean-Baptiste Corot Savigny-sur-Orge

### Translation and Interpretation

 facilitated communication between French and English speakers. 

## Certification in Double Dutch Judging, Federation Française de Double Dutch (FFDD), 2020

### Sports Event Planning and Management

Assisted with organizing local Double Dutch competitions and judged during events. 

## Master of Licence économie et gestion, UPE C & M Créteil

### Economic Research Intern, Institute for Economic Research (IEP), 2023 

## Licenciée en Microéconomie, Macroéconomie, Processus Stochastique, UPE C & M Créteil

### Volunteer, Refugee Support Organization, 2022

Please note that this draft is just a starting point. You may want to customize and expand each section with more details, particularly the "Work Experience" and "Projects/Internships/Volunteer Experience" parts. Additionally, you can format and organize these sections according to your preferences or resume standards specific to your country or industry. 

In [127]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import CohereEmbeddings
import streamlit as st  # Import Streamlit pour afficher en Markdown

def generate_cv2(query, vectorstore, cohere_api_key, temperature=0.4, max_tokens=2000):
    """
    Utilise un LLM pour générer un CV basé sur la requête et les documents,
    avec des options de personnalisation pour la température et le nombre de tokens.
    """
    # Créer un retriever pour interroger le vectorstore
    retriever = vectorstore.as_retriever()

    # Définir le prompt qui sera utilisé pour la génération du texte
    prompt = PromptTemplate(
        input_variables=["context", "query"],
        template=(
            "En se basant sur les informations suivantes : {context}\n"
            "Répondez à la requête suivante : {query}"
        )
    )  # Ajouter le user_agent
    user_agent = "mon_client"  # Choisissez un nom pour le client

    # Créer les embeddings avec le user_agent
    cohere_embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, user_agent=user_agent)

    # Créer le modèle Cohere pour la génération de texte
    llm = Cohere(
        cohere_api_key=cohere_api_key,  # Utilisation de la clé API directement ici
        model="command-xlarge",  # Spécifiez le modèle de génération approprié
        temperature=temperature,
        max_tokens=max_tokens
    )
    # Créer la chaîne de question-réponse avec les paramètres de température et de tokens
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Cette méthode combine les documents
        retriever=retriever,
        return_source_documents=False
    )
    # Exécuter la chaîne pour obtenir la réponse
    response = qa_chain.run(query)

    return response

# Exemple d'utilisation de la fonction avec Streamlit
if __name__ == "__main__":
    # Configurer la clé API Cohere
    COHERE_API_KEY = "IiVPAkCJc5XsF4abBihOzkj2I7WcYKVH4KgLZMID"

    # Titre de l'application
    st.title("Générateur de CV")

    # Créer un vectorstore avec FAISS et CohereEmbeddings
    embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY, user_agent="mon_client")
    vectorstore = FAISS.from_documents(documents3, embeddings)

    # Générer un CV basé sur une requête utilisateur avec des paramètres de température et de tokens
    query = "Créer un CV basé sur mes expériences et certificats"
    generated_cv = generate_cv2(query, vectorstore, COHERE_API_KEY, temperature=0.7, max_tokens=2000)

    # Afficher le CV généré dans un conteneur Markdown
    st.markdown("## CV Généré")
    st.markdown(generated_cv)  # Le CV est affiché en Markdown


2024-12-08 00:56:11.962 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 00:56:11.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 00:56:56.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 00:56:56.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 00:56:56.887 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 00:56:56.887 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [139]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import CohereEmbeddings
from IPython.display import display, Markdown

def generate_cv2(query, vectorstore, cohere_api_key, temperature=0.4, max_tokens=2000):
    """
    Utilise un LLM pour générer un CV basé sur la requête et les documents,
    avec des options de personnalisation pour la température et le nombre de tokens.
    """
    # Créer un retriever pour interroger le vectorstore
    retriever = vectorstore.as_retriever()

    # Définir le prompt qui sera utilisé pour la génération du texte
    prompt = PromptTemplate(
        input_variables=["context", "query"],
        template=(
            "En se basant sur les informations suivantes : {context}\n"
            "Répondez à la requête suivante : {query}"
        )
    )

    # Ajouter le user_agent
    user_agent = "mon_client"  # Choisissez un nom pour le client

    # Créer les embeddings avec le user_agent
    cohere_embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, user_agent=user_agent)

    # Créer le modèle Cohere pour la génération de texte
    llm = Cohere(
        cohere_api_key=cohere_api_key,  # Utilisation de la clé API directement ici
        model="command-xlarge",  # Spécifiez le modèle de génération approprié
        temperature=temperature,
        max_tokens=max_tokens
    )

    # Créer la chaîne de question-réponse avec les paramètres de température et de tokens
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Cette méthode combine les documents
        retriever=retriever,
        return_source_documents=False
    )
    
    # Exécuter la chaîne pour obtenir la réponse
    response = qa_chain.run(query)

    return response


# Exemple d'utilisation de la fonction avec des paramètres modifiables
if __name__ == "__main__":
    # Configurer la clé API Cohere
    COHERE_API_KEY = "IiVPAkCJc5XsF4abBihOzkj2I7WcYKVH4KgLZMID"


    embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY, user_agent="mon_client")
    vectorstore = FAISS.from_documents(documents3, embeddings)

    # Générer un CV basé sur une requête utilisateur avec des paramètres de température et de tokens
    query = '''Créer un CV basé sur mes expériences et certificats en respectant ce plan :
    1- Information 
    2 - Education 
    3- Certifications
    4 - Skills
    5 - Work Experience
    6 - Projects/Internships/Volunteer Experience '''
    generated_cv = generate_cv2(query, vectorstore, COHERE_API_KEY, temperature=0.7, max_tokens=2000)

    # Afficher le CV généré en Markdown
    print("\n### CV Généré :\n")
    markdown_cv = f"""
    # Curriculum Vitae

    {generated_cv}
    """
    display(Markdown(generated_cv))


### CV Généré :



 Sure, I can help you create a basic CV based on the information you've provided. Please note that a CV for a French context may differ from that in other countries, so this may not be a perfect match for what you are seeking. Here is a draft of a CV following the structure you require: 

---
**Information**
- Name: Assa Traore
- Address: Grigny (91350) 
- Phone: Ile-de-France 07 69 88 07 32
- Email: assa16@gmail.com
- Date of Birth: 16/01/2004

**Education**
- Master MASERA TI UPE C, Data Analysis, Créteil
- Licence économie e t gestion, UPE C, Créteil
- Lycée Jean-Baptiste Corot, Savigny-sur -Orge, Baccalaur éat, Maths - SES - HGGSP

**Certifications**
- Certificate of Formation as a Judge for Double Dutch, December 2020. 
- Language certifications (TOEIC: 731/990, Español)

**Skills**
- Data analysis
- Software and language programming (Logique e t programma tion)
- Algorithme a vancée
- Economic analysis (Microéconomie Macroéconomie)
- Communication (TOEIC: 731/990)
- Leadership 
- Event organization 

**Work Experience**
- Aide aux de voirs, Maire de G rigny, Grigny. 
- Apprentissage des leçons, Aide à l'apprentissage des leçons, Hôtesse de caisse Bricoman Monthléry.
- Inventaire et contrôle des niv eaux des st ocks, Applica tion des pr océdures de f erme ture, Brétigny-sur -Orge.
- Conseille clien tèle, Gestion des c ommandes driv e, Sonink é, Langue ma ternelle Anglais. 

**Projects/Internships/Volunteer Experience**
- Mise en plac e d'événemen ts sportifs e t cultur elles pour les en fants n'ayant pas pu aller en vacanc es à cause du c ontexte pandémique de cet été, Grigny. 
- Accompagnemen t des élè ves dans leur s révisions, Propositions d' exercices supplémen taires, Aide à l'apprentissag e des leçons, Équipièr e polyv alente McDonald's, Grigny. 

--- 
Please note that this is just a starting point and you may want to edit and expand on these points to create a more detailed and personalised CV. 
Let me know if you would like me to proceed to the next steps and add more information or edit the existing copy. 
I can also advice on how to include specific information in a CV for French contexts if this is what you are seeking. Do you need advice for further sections or would you like me to start removing identifying information for privacy? 